In [7]:
# from multiprocessing import Pool
import multiprocessing
import ASWmultiprocess as PC

def dummy_func(chunk):
    global ready_list
    ready_list.append(chunk)
    
if __name__ == "__main__":
    
    # "Кнопки"
#     t_air_list = [-29,-15, 0, 8, 15] # - температуры, при которых необходимо расчитать
    t_air_list = [-29,-25,-20, -15, -10,-5,0,5] # - температуры, при которых необходимо расчитать
#     t_air_list = [-29] # - температуры, при которых необходимо расчитать
    s = len(t_air_list)# - длина списка температур
    nagr_list = 1# -нагрузки, при которых необходимо расчитать

    # Флаги
    ASWbul = [0,1,2] #- 0- нет аккум 1- загрузка аккумулятора, 2- разгрузка аккумулятора
#     ASWbul = [0]
    sn = len(ASWbul)
    ASWatm = [False,True]  #- True- атмосферный аккум False- аккум под давлением
#     ASWatm = [False]  #- True- атмосферный аккум False- аккум под давлением
    sz = len(ASWatm)
    PKM_zaryad = False
    Сalculate_minimum = [False,True]
#     Сalculate_minimum = [False]
    sk = len(Сalculate_minimum)
    Teplo = int(True)
    # Вкл выкл заряд по ПКМ
#     PKM_zaryad = False
    
    #ASW konstruct parametr
    Diametr = 4
    Visota=10
    kolichestvo=2
    lambda_min_vata = 0.045
    delta_min_vata = 0.1
    vremya=4
    vremyaojidaniya = 12

    # Основные эффективности оборудования
    KPD_PN = 0.8074
    KPD_KN = 0.75
    KPD_to = 0.99
    KPD_SP = 0.99

    # Параметры, отвечающие за процесс расчета
    Calcmethod = "hybr"
    Calctolerance = 10**-2
    Maxiterations_KU_TU = 15
    Maxiterations_cotel = 5
    Maxiterations_turbine = 15
    
    steamVD_fraction_to_turbine=1
    
    #####################
    for k in range(sk):
        for z in range(sz):
            print("Начался расчет Сalculate_minimum=",Сalculate_minimum[k],'ASWatm=', ASWatm[z])
            chunks = [
                (
                    Diametr,  # Diametr - диаметр аккумулятора
                    kolichestvo,  # kolichestvo -  количество баков аккумуляторов
                    Visota,  # Visota - высота аккумулятора
                    lambda_min_vata,  # - lambda_min_vata теплопроводность изолятора бака аккумулятора
                    delta_min_vata,  # - delta_min_vata толщина изолятора бака аккумулятора
                    ASWbul[i],  # ASWbul - 1- загрузка аккумулятора, 2- разгрузка аккумулятора
                    nagr_list,  # nagr - доля нагрузки от номинала
                    vremya,  # vremya - время зарядки/разрядки аккумулятора
                    vremyaojidaniya,  # vremyaojidaniya - время ожидания аккумулятора
                    t_air_list[j],
                    PKM_zaryad,
                    Сalculate_minimum[k],
                    KPD_PN,
                    KPD_KN,
                    KPD_to,
                    KPD_SP,
                    Calcmethod,
                    Calctolerance,
                    Maxiterations_KU_TU,
                    Maxiterations_cotel,
                    Maxiterations_turbine,
                    Сalculate_minimum[k],
                    Teplo,
                    steamVD_fraction_to_turbine,
                    PKM_zaryad,
                    ASWatm[z],
                )
                for i in range(sn)
                for j in range(s)
            ]

            pool = Pool(processes=sn * s)
            results = pool.map(PC.ParallelCompute, chunks)
            # clean up
            pool.close()
            pool.join()

            import pandas as pd
            import openpyxl
            import os

            
            print(result)
            df = pd.DataFrame(all_results)
            df = pd.DataFrame(result)
            df = df.T

            if ASWatm[z]==False and Сalculate_minimum[k]==False:
                book = openpyxl.load_workbook('resdataASW.xlsx')
                writer = pd.ExcelWriter('resdataASW.xlsx', engine='openpyxl') 
                writer.book = book
                writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
                df = pd.DataFrame(result)
#                 df = pd.DataFrame(all_results)
                df.to_excel(writer, "Максимум", startrow=0, startcol=0)
                writer.save()
            if ASWatm[z]==True and Сalculate_minimum[k]==False:
                book = openpyxl.load_workbook('resdataASW.xlsx')
                writer = pd.ExcelWriter('resdataASW.xlsx', engine='openpyxl') 
                writer.book = book
                writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
                df = pd.DataFrame(result)
                df.to_excel(writer, "Максимуматм", startrow=0, startcol=0)
                writer.save()
            if ASWatm[z]==True and Сalculate_minimum[k]==True:
                book = openpyxl.load_workbook('resdataASW.xlsx')
                writer = pd.ExcelWriter('resdataASW.xlsx', engine='openpyxl') 
                writer.book = book
                writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
                df = pd.DataFrame(result)
                df.to_excel(writer, "Минимуматм", startrow=0, startcol=0)
                writer.save()
            if ASWatm[z]==False and Сalculate_minimum[k]==True:
                book = openpyxl.load_workbook('resdataASW.xlsx')
                writer = pd.ExcelWriter('resdataASW.xlsx', engine='openpyxl') 
                writer.book = book
                writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
                df = pd.DataFrame(result)
                df.to_excel(writer, "Минимум", startrow=0, startcol=0)
                writer.save()

Начался расчет Сalculate_minimum= False ASWatm= False
Начался расчет Сalculate_minimum= False ASWatm= True
Начался расчет Сalculate_minimum= True ASWatm= False
Начался расчет Сalculate_minimum= True ASWatm= True
